In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.linalg as linalg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

exp = np.exp
pdf = stats.norm.pdf
cdf = stats.norm.cdf
sqrt = np.sqrt
log = np.log

def d1(f,k,t,r,sigma):
    return log(f/k)/sigma/sqrt(t) + 0.5 * sigma * sqrt(t)
    
def d2(f,k,t,r,sigma):
    return log(f/k)/sigma/sqrt(t) - 0.5 * sigma * sqrt(t)
    
def price(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    d2_ = d2(f,k,t,r,sigma)
    if types == "c":
        return exp(-r*t)*(f*cdf(d1_) - k*cdf(d2_))
    elif types == "p":
        return exp(-r*t)*(k*cdf(-d2_) - f*cdf(-d1_))

def volatility(f,k,t,r,market_price,types):
    left = 0.0001
    right = 2
    mid = 0
    for i in range(20):
        mid = (left + right)/2
        theoretical_price = price(f,k,t,r,mid,types)
        if theoretical_price < market_price:
            left = mid
        else:
            right = mid
    return mid

def delta(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    if types=="c":
        return exp(-r*t)*cdf(d1_)
    elif types=="p":
        return -exp(-r*t)*cdf(-d1_)
    
def vega(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    return f*exp(-r*t)*pdf(d1_) * sqrt(t) * 0.01

def theta(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    d2_ = d2(f,k,t,r,sigma)
    if types== "c":
        return (-f*exp(-r*t)*pdf(d1_)*sigma/2/sqrt(t) - r * k * exp(-r*t)*cdf(d2_) + r * f * exp(-r*t) * cdf(d1_))/365
    elif types == "p":
        return (-f*exp(-r*t)*pdf(d1_)*sigma/2/sqrt(t) + r * k * exp(-r*t)*cdf(-d2_) - r * f * exp(-r*t) * cdf(-d1_))/365 

def rho(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    d2_ = d2(f,k,t,r,sigma)
    if types=="c":
        return -t*exp(-r*t)*(f*cdf(d1_) - k*cdf(d2_))*0.01
    elif types == "p":
        return -t*exp(-r*t)*(k*cdf(-d2_) - f*cdf(-d1_))*0.01

def gamma(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    return exp(-r*t)*pdf(d1_)/f/sigma/sqrt(t)

def vanna(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    return vega(f,k,t,r,sigma,types) / f * ( 1 - d1_/sigma/sqrt(t) )

def vomma(f,k,t,r,sigma,types):
    d1_ = d1(f,k,t,r,sigma)
    d2_ = d2(f,k,t,r,sigma)
    return vega(f,k,t,r,sigma,types) * d1_*d2_/sigma*0.01

greeks = [delta, vega, theta, rho, gamma, vanna, vomma]


f,k,t,r,sigma,types = 364.1, 357.5, 0.0411, 0.015, 0.2516, 'c'

# for func in greeks:
#     print(func(k,k,t,r,sigma,types))





#######################################################################################################################################################



def back_test1(strike, the_date,margin1,margin2):
    
    pd.set_option('display.max_row',500)
    pd.set_option('display.max_column', 100)
    
    #cd_rate = 1.5
    #current_date = date(int(the_date[0:4]),int(the_date[4:6]),int(the_date[6:8]))
    #maturity_date = date(2022,3,10)
    #days_left = (maturity_date - current_date).days
    #discount = 1/(1 + cd_rate/100 * (days_left)/365)
    #multiplier = 250000
    
    c = 'c'
    p = 'p'
    f = 'f'
    
    name_list = [c,p,f]
    
    df = pd.read_csv(r'C:\DTS\DTS\report\{}.csv'.format(the_date +'_index_'+ str(strike)),header = None,
                     usecols = [1,2,6,7,8,9,10,11])
    df.columns = ['time', 'code', 'askc','askv','askp','bidp','bidv','bidc']
    df = df[ (df.time > '09:00:00-100') & ( df.time < '15:35:00-000') ]
    tmp_code_list = list(df.code.unique())
    tmp_code_list.sort()
    f_code, c_code, p_code = tmp_code_list
    code_list = [c_code, p_code, f_code]
    if strike%5!=0: strike += 0.5
    
    df.sort_values('time',inplace=True)
    df.index = range(len(df))
    df.time = pd.to_timedelta(df.time.str.replace('-','.'))
    df = df.replace(' ',0)
    df.code = df.code.astype('string')
    
    df.askc = df.askc.astype('float')
    df.askv = df.askv.astype('float')
    df.bidv = df.bidv.astype('float')
    df.bidc = df.bidc.astype('float')
    
    df['type'] = ''
    df.type = np.where( ( df.askv == 0 ) , 'ask side exec', df.type)
    df.type = np.where( ( df.bidv == 0 ) , 'bid side exec', df.type)
    df.type = np.where( (df.askc==0) & (df.bidc==0), 'price change', df.type )
    df.type = np.where( (df.type==''), 'volume change', df.type)
    
    df['exec_volume'] = np.nan
    df['exec_volume'] = np.where( df.type == 'ask side exec', df.bidc, df['exec_volume'] )
    df['exec_volume'] = np.where( df.type == 'bid side exec', df.askc, df['exec_volume'] )
    
    df['exec_price'] = np.nan
    df['exec_price'] = np.where(df.type == 'ask side exec', df.bidv, df['exec_price'])
    df['exec_price'] = np.where(df.type == 'bid side exec', df.askv, df['exec_price'])
    
    
    
    for name,code in zip(name_list,code_list):
        
        df[name+'askv'] = np.nan
        df[name+'askv'] = np.where((df.askv!=0) & (df.code == code) & 
                               (df.type != 'ask side exec') & (df.type != 'bid side exec') ,
                               df['askv'],df[name+'askv'])
        df[name+'askv'] = df[name+'askv'].ffill()
        
        df[name+'askp'] = np.nan
        df[name+'askp'] = np.where((df.askp!=0) & (df.code == code),df['askp'],df[name+'askp'])
        df[name+'askp'] = df[name+'askp'].ffill()
        
        df[name+'bidp'] = np.nan
        df[name+'bidp'] = np.where((df.bidp!=0) & (df.code == code),df['bidp'],df[name+'bidp'])
        df[name+'bidp'] = df[name+'bidp'].ffill()
        
        df[name+'bidv'] = np.nan
        df[name+'bidv'] = np.where((df.bidv!=0) & (df.code == code)  &
                               (df.type != 'ask side exec') & (df.type != 'bid side exec'),
                               df['bidv'],df[name+'bidv'])
        df[name+'bidv'] = df[name+'bidv'].ffill()
    
    df.exec_price.fillna(0,inplace = True)
    df.exec_volume.fillna(0, inplace = True)
    df.dropna(inplace = True)
    
    for name in name_list:
        
        df[name + 'askup'] = ''
        df[name + 'biddown'] = ''
        
        df[name + 'askup'] = np.where(df[name+'askp'] > df[name+'askp'].shift(1), 'O','')
        df[name + 'biddown'] = np.where(df[name + 'bidp'] < df[name+'bidp'].shift(1), 'O','')
    
    for name in name_list:
        #df[name+'hold'] = 0
        df[name+'buy_price'] = 0.0
        df[name+'sell_price'] = 0.0
        
        
    # df['c_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.cbidp, pd.Series(["c"]*len(df)))
    # df['p_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.pbidp, pd.Series(["p"]*len(df)))
    
    
    # df['delta'] = delta(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)),pd.Series([0.2]*len(df)), pd.Series(["p"]*len(df)))
    
    
    df['spread1'] = 0
    df['spread2'] = 0
    
    df.spread1 = df.faskp - df.cbidp + df.paskp - strike
    df.spread2 = df.fbidp - df.caskp + df.pbidp - strike
    
    cnt1 = len(df[df.spread1<=-margin1])
    cnt2 = len(df[df.spread2>=margin2])
    
    print(round((strike - 350)/2.5), ' date: ',the_date, 'strike: ',strike,'cnt1: ',cnt1, 'cnt2: ',cnt2, 'paskp: ', \
          round(df.paskp.mean(),2), 'pbidp: ', round(df.pbidp.mean(),2), 'caskp: ', round(df.caskp.mean(),2), 'cbidp: ', \
              round(df.cbidp.mean(),2), 'spread1: ', round(df.spread1.mean(),2), 'spread2: ', round(df.spread2.mean(),2))
    return [df, strike, the_date, cnt1, cnt2]


####################################################################################################################################




def back_test2(strike, the_date,margin1,margin2):
    
    pd.set_option('display.max_row',500)
    pd.set_option('display.max_column', 100)
    
    #cd_rate = 1.5
    #current_date = date(int(the_date[0:4]),int(the_date[4:6]),int(the_date[6:8]))
    #maturity_date = date(2022,3,10)
    #days_left = (maturity_date - current_date).days
    #discount = 1/(1 + cd_rate/100 * (days_left)/365)
    #multiplier = 250000
    
    c = 'c'
    p = 'p'
    f = 'f'
    
    name_list = [c,p,f]
    
    df = pd.read_csv(r'C:\DTS\DTS\report\{}.csv'.format(the_date +'_index_'+ str(strike)),header = None,
                     usecols = [1,2,6,7,8,9,10,11])
    df.columns = ['time', 'code', 'askc','askv','askp','bidp','bidv','bidc']
    df = df[ (df.time > '09:00:00-100') & ( df.time < '15:35:00-000') ]
    tmp_code_list = list(df.code.unique())
    tmp_code_list.sort()
    f_code, c_code, p_code = tmp_code_list
    code_list = [c_code, p_code, f_code]
    if strike%5!=0: strike += 0.5
    
    df.sort_values('time',inplace=True)
    df.index = range(len(df))
    df.time = pd.to_timedelta(df.time.str.replace('-','.'))
    df = df.replace(' ',0)
    df.code = df.code.astype('string')
    
    df.askc = df.askc.astype('float')
    df.askv = df.askv.astype('float')
    df.bidv = df.bidv.astype('float')
    df.bidc = df.bidc.astype('float')
    
    df['type'] = ''
    df.type = np.where( ( df.askv == 0 ) , 'ask side exec', df.type)
    df.type = np.where( ( df.bidv == 0 ) , 'bid side exec', df.type)
    df.type = np.where( (df.askc==0) & (df.bidc==0), 'price change', df.type )
    df.type = np.where( (df.type==''), 'volume change', df.type)
    
    df['exec_volume'] = np.nan
    df['exec_volume'] = np.where( df.type == 'ask side exec', df.bidc, df['exec_volume'] )
    df['exec_volume'] = np.where( df.type == 'bid side exec', df.askc, df['exec_volume'] )
    
    df['exec_price'] = np.nan
    df['exec_price'] = np.where(df.type == 'ask side exec', df.bidv, df['exec_price'])
    df['exec_price'] = np.where(df.type == 'bid side exec', df.askv, df['exec_price'])
    
    
    
    for name,code in zip(name_list,code_list):
        
        df[name+'askv'] = np.nan
        df[name+'askv'] = np.where((df.askv!=0) & (df.code == code) & 
                               (df.type != 'ask side exec') & (df.type != 'bid side exec') ,
                               df['askv'],df[name+'askv'])
        df[name+'askv'] = df[name+'askv'].ffill()
        
        df[name+'askp'] = np.nan
        df[name+'askp'] = np.where((df.askp!=0) & (df.code == code),df['askp'],df[name+'askp'])
        df[name+'askp'] = df[name+'askp'].ffill()
        
        df[name+'bidp'] = np.nan
        df[name+'bidp'] = np.where((df.bidp!=0) & (df.code == code),df['bidp'],df[name+'bidp'])
        df[name+'bidp'] = df[name+'bidp'].ffill()
        
        df[name+'bidv'] = np.nan
        df[name+'bidv'] = np.where((df.bidv!=0) & (df.code == code)  &
                               (df.type != 'ask side exec') & (df.type != 'bid side exec'),
                               df['bidv'],df[name+'bidv'])
        df[name+'bidv'] = df[name+'bidv'].ffill()
    
    df.exec_price.fillna(0,inplace = True)
    df.exec_volume.fillna(0, inplace = True)
    df.dropna(inplace = True)
    
    for name in name_list:
        
        df[name + 'askup'] = ''
        df[name + 'biddown'] = ''
        
        df[name + 'askup'] = np.where(df[name+'askp'] > df[name+'askp'].shift(1), 'O','')
        df[name + 'biddown'] = np.where(df[name + 'bidp'] < df[name+'bidp'].shift(1), 'O','')
    
    for name in name_list:
        #df[name+'hold'] = 0
        df[name+'buy_price'] = 0.0
        df[name+'sell_price'] = 0.0
        
        
    # df['c_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.cbidp, pd.Series(["c"]*len(df)))
    # df['p_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.pbidp, pd.Series(["p"]*len(df)))
    
    
    # df['delta'] = delta(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)),pd.Series([0.2]*len(df)), pd.Series(["p"]*len(df)))
    
    
    df['spread1'] = 0
    df['spread2'] = 0
    
    df.spread1 = df.exec_price - df.cbidp + df.paskp - strike
    df.spread2 = df.exec_price - df.caskp + df.pbidp - strike
    df = df[((df.type == 'ask side exec') | (df.type == 'bid side exec')) & (df.code == f_code) ]
    cnt1 = len(df[(df.spread1<=-margin1) & (df.type == 'bid side exec')])
    cnt2 = len(df[(df.spread2>=margin2) & (df.type == 'ask side exec')])
    
    print(round((strike - 350)/2.5), ' date: ',the_date, 'strike: ',strike,'cnt1: ',cnt1, 'cnt2: ',cnt2, 'paskp: ', \
          round(df.paskp.mean(),2), 'pbidp: ', round(df.pbidp.mean(),2), 'caskp: ', round(df.caskp.mean(),2), 'cbidp: ', \
              round(df.cbidp.mean(),2), 'spread1: ', round(df.spread1.mean(),2), 'spread2: ', round(df.spread2.mean(),2))
    return [df, strike, the_date, cnt1, cnt2]






#########################################################################################################################


def back_test3(strike, the_date,margin1,margin2):
    
    pd.set_option('display.max_row',500)
    pd.set_option('display.max_column', 100)
    
    #cd_rate = 1.5
    #current_date = date(int(the_date[0:4]),int(the_date[4:6]),int(the_date[6:8]))
    #maturity_date = date(2022,3,10)
    #days_left = (maturity_date - current_date).days
    #discount = 1/(1 + cd_rate/100 * (days_left)/365)
    #multiplier = 250000
    
    c = 'c'
    p = 'p'
    f = 'f'
    
    name_list = [c,p,f]
    
    df = pd.read_csv(r'C:\DTS\DTS\report\{}.csv'.format(the_date +'_index_'+ str(strike)),header = None,
                     usecols = [1,2,6,7,8,9,10,11])
    df.columns = ['time', 'code', 'askc','askv','askp','bidp','bidv','bidc']
    df = df[ (df.time > '09:00:00-100') & ( df.time < '15:35:00-000') ]
    tmp_code_list = list(df.code.unique())
    tmp_code_list.sort()
    f_code, c_code, p_code = tmp_code_list
    code_list = [c_code, p_code, f_code]
    if strike%5!=0: strike += 0.5
    
    df.sort_values('time',inplace=True)
    df.index = range(len(df))
    df.time = pd.to_timedelta(df.time.str.replace('-','.'))
    df = df.replace(' ',0)
    df.code = df.code.astype('string')
    
    df.askc = df.askc.astype('float')
    df.askv = df.askv.astype('float')
    df.bidv = df.bidv.astype('float')
    df.bidc = df.bidc.astype('float')
    
    df['type'] = ''
    df.type = np.where( ( df.askv == 0 ) , 'ask side exec', df.type)
    df.type = np.where( ( df.bidv == 0 ) , 'bid side exec', df.type)
    df.type = np.where( (df.askc==0) & (df.bidc==0), 'price change', df.type )
    df.type = np.where( (df.type==''), 'volume change', df.type)
    
    df['exec_volume'] = np.nan
    df['exec_volume'] = np.where( df.type == 'ask side exec', df.bidc, df['exec_volume'] )
    df['exec_volume'] = np.where( df.type == 'bid side exec', df.askc, df['exec_volume'] )
    
    df['exec_price'] = np.nan
    df['exec_price'] = np.where(df.type == 'ask side exec', df.bidv, df['exec_price'])
    df['exec_price'] = np.where(df.type == 'bid side exec', df.askv, df['exec_price'])
    
    
    
    for name,code in zip(name_list,code_list):
        
        df[name+'askv'] = np.nan
        df[name+'askv'] = np.where((df.askv!=0) & (df.code == code) & 
                               (df.type != 'ask side exec') & (df.type != 'bid side exec') ,
                               df['askv'],df[name+'askv'])
        df[name+'askv'] = df[name+'askv'].ffill()
        
        df[name+'askp'] = np.nan
        df[name+'askp'] = np.where((df.askp!=0) & (df.code == code),df['askp'],df[name+'askp'])
        df[name+'askp'] = df[name+'askp'].ffill()
        
        df[name+'bidp'] = np.nan
        df[name+'bidp'] = np.where((df.bidp!=0) & (df.code == code),df['bidp'],df[name+'bidp'])
        df[name+'bidp'] = df[name+'bidp'].ffill()
        
        df[name+'bidv'] = np.nan
        df[name+'bidv'] = np.where((df.bidv!=0) & (df.code == code)  &
                               (df.type != 'ask side exec') & (df.type != 'bid side exec'),
                               df['bidv'],df[name+'bidv'])
        df[name+'bidv'] = df[name+'bidv'].ffill()
    
    df.exec_price.fillna(0,inplace = True)
    df.exec_volume.fillna(0, inplace = True)
    df.dropna(inplace = True)
    
    for name in name_list:
        
        df[name + 'askup'] = ''
        df[name + 'biddown'] = ''
        
        df[name + 'askup'] = np.where(df[name+'askp'] > df[name+'askp'].shift(1), 'O','')
        df[name + 'biddown'] = np.where(df[name + 'bidp'] < df[name+'bidp'].shift(1), 'O','')
    
    for name in name_list:
        #df[name+'hold'] = 0
        df[name+'buy_price'] = 0.0
        df[name+'sell_price'] = 0.0
        
        
    # df['c_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.cbidp, pd.Series(["c"]*len(df)))
    # df['p_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.pbidp, pd.Series(["p"]*len(df)))
    
    
    # df['delta'] = delta(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)),pd.Series([0.2]*len(df)), pd.Series(["p"]*len(df)))
    
    
    df['spread1'] = 0
    df['spread2'] = 0
    
    df.spread1 = df.faskp - df.exec_price + df.paskp - strike
    df.spread2 = df.fbidp - df.exec_price + df.pbidp - strike
    df = df[((df.type == 'ask side exec') | (df.type == 'bid side exec')) & (df.code == c_code) ]
    cnt1 = len(df[(df.spread1<=-margin1) & (df.type == 'ask side exec')])
    cnt2 = len(df[(df.spread2>=margin2) & (df.type == 'bid side exec')])
    
    print(round((strike - 350)/2.5), ' date: ',the_date, 'strike: ',strike,'cnt1: ',cnt1, 'cnt2: ',cnt2, 'paskp: ', \
          round(df.paskp.mean(),2), 'pbidp: ', round(df.pbidp.mean(),2), 'caskp: ', round(df.caskp.mean(),2), 'cbidp: ', \
              round(df.cbidp.mean(),2), 'spread1: ', round(df.spread1.mean(),2), 'spread2: ', round(df.spread2.mean(),2))
    return [df, strike, the_date, cnt1, cnt2]






#########################################################################################################################


def back_test4(strike, the_date,margin1,margin2):
    
    pd.set_option('display.max_row',500)
    pd.set_option('display.max_column', 100)
    
    #cd_rate = 1.5
    #current_date = date(int(the_date[0:4]),int(the_date[4:6]),int(the_date[6:8]))
    #maturity_date = date(2022,3,10)
    #days_left = (maturity_date - current_date).days
    #discount = 1/(1 + cd_rate/100 * (days_left)/365)
    #multiplier = 250000
    
    c = 'c'
    p = 'p'
    f = 'f'
    
    name_list = [c,p,f]
    
    df = pd.read_csv(r'C:\DTS\DTS\report\{}.csv'.format(the_date +'_index_'+ str(strike)),header = None,
                     usecols = [1,2,6,7,8,9,10,11])
    df.columns = ['time', 'code', 'askc','askv','askp','bidp','bidv','bidc']
    df = df[ (df.time > '09:00:00-100') & ( df.time < '15:35:00-000') ]
    tmp_code_list = list(df.code.unique())
    tmp_code_list.sort()
    f_code, c_code, p_code = tmp_code_list
    code_list = [c_code, p_code, f_code]
    if strike%5!=0: strike += 0.5
    
    df.sort_values('time',inplace=True)
    df.index = range(len(df))
    df.time = pd.to_timedelta(df.time.str.replace('-','.'))
    df = df.replace(' ',0)
    df.code = df.code.astype('string')
    
    df.askc = df.askc.astype('float')
    df.askv = df.askv.astype('float')
    df.bidv = df.bidv.astype('float')
    df.bidc = df.bidc.astype('float')
    
    df['type'] = ''
    df.type = np.where( ( df.askv == 0 ) , 'ask side exec', df.type)
    df.type = np.where( ( df.bidv == 0 ) , 'bid side exec', df.type)
    df.type = np.where( (df.askc==0) & (df.bidc==0), 'price change', df.type )
    df.type = np.where( (df.type==''), 'volume change', df.type)
    
    df['exec_volume'] = np.nan
    df['exec_volume'] = np.where( df.type == 'ask side exec', df.bidc, df['exec_volume'] )
    df['exec_volume'] = np.where( df.type == 'bid side exec', df.askc, df['exec_volume'] )
    
    df['exec_price'] = np.nan
    df['exec_price'] = np.where(df.type == 'ask side exec', df.bidv, df['exec_price'])
    df['exec_price'] = np.where(df.type == 'bid side exec', df.askv, df['exec_price'])
    
    
    
    for name,code in zip(name_list,code_list):
        
        df[name+'askv'] = np.nan
        df[name+'askv'] = np.where((df.askv!=0) & (df.code == code) & 
                               (df.type != 'ask side exec') & (df.type != 'bid side exec') ,
                               df['askv'],df[name+'askv'])
        df[name+'askv'] = df[name+'askv'].ffill()
        
        df[name+'askp'] = np.nan
        df[name+'askp'] = np.where((df.askp!=0) & (df.code == code),df['askp'],df[name+'askp'])
        df[name+'askp'] = df[name+'askp'].ffill()
        
        df[name+'bidp'] = np.nan
        df[name+'bidp'] = np.where((df.bidp!=0) & (df.code == code),df['bidp'],df[name+'bidp'])
        df[name+'bidp'] = df[name+'bidp'].ffill()
        
        df[name+'bidv'] = np.nan
        df[name+'bidv'] = np.where((df.bidv!=0) & (df.code == code)  &
                               (df.type != 'ask side exec') & (df.type != 'bid side exec'),
                               df['bidv'],df[name+'bidv'])
        df[name+'bidv'] = df[name+'bidv'].ffill()
    
    df.exec_price.fillna(0,inplace = True)
    df.exec_volume.fillna(0, inplace = True)
    df.dropna(inplace = True)
    
    for name in name_list:
        
        df[name + 'askup'] = ''
        df[name + 'biddown'] = ''
        
        df[name + 'askup'] = np.where(df[name+'askp'] > df[name+'askp'].shift(1), 'O','')
        df[name + 'biddown'] = np.where(df[name + 'bidp'] < df[name+'bidp'].shift(1), 'O','')
    
    for name in name_list:
        #df[name+'hold'] = 0
        df[name+'buy_price'] = 0.0
        df[name+'sell_price'] = 0.0
        
        
    # df['c_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.cbidp, pd.Series(["c"]*len(df)))
    # df['p_volatility'] = volatility(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)), df.pbidp, pd.Series(["p"]*len(df)))
    # df['delta'] = delta(df.fbidp, pd.Series([strike]*len(df)), pd.Series([days_left/365]*len(df)),pd.Series([cd_rate]*len(df)),pd.Series([0.2]*len(df)), pd.Series(["p"]*len(df)))
    
    
    df['spread1'] = 0
    df['spread2'] = 0
    
    df.spread1 = df.faskp - df.cbidp + df.exec_price - strike
    df.spread2 = df.fbidp - df.caskp + df.exec_price - strike
    df = df[((df.type == 'ask side exec') | (df.type == 'bid side exec')) & (df.code == p_code) ]
    cnt1 = len(df[(df.spread1<=-margin1) & (df.type == 'bid side exec')])
    cnt2 = len(df[(df.spread2>=margin2)& (df.type == 'ask side exec')])
    
    print(round((strike - 350)/2.5), ' date: ',the_date, 'strike: ',strike,'cnt1: ',cnt1, 'cnt2: ',cnt2, 'paskp: ', \
          round(df.paskp.mean(),2), 'pbidp: ', round(df.pbidp.mean(),2), 'caskp: ', round(df.caskp.mean(),2), 'cbidp: ', \
              round(df.cbidp.mean(),2), 'spread1: ', round(df.spread1.mean(),2), 'spread2: ', round(df.spread2.mean(),2))
    return [df, strike, the_date, cnt1, cnt2]






#########################################################################################################################




def back_test5(strike, the_date, strategy,start, margin1, margin2):
    
    pd.set_option('display.max_row',500)
    pd.set_option('display.max_column', 100)
    
    # cd_rate = 1.5
    # current_date = date(int(the_date[0:4]),int(the_date[4:6]),int(the_date[6:8]))
    # maturity_date = date(2022,3,10)
    # days_left = (maturity_date - current_date).days
    # discount = 1/(1 + cd_rate/100 * (days_left)/365)
    
    multiplier = 250000
    
    c = 'c'
    p = 'p'
    f = 'f'

    name_list = [c,p,f]
    
    df = pd.read_csv(r'C:\DTS\DTS\report\{}.csv'.format(the_date +'_index_'+ str(strike)),header = None,
                     usecols = [1,2,6,7,8,9,10,11])
    df.columns = ['time', 'code', 'askc','askv','askp','bidp','bidv','bidc']
    df = df[ (df.time > '09:00:01-000') & ( df.time < '15:35:00-000') ]
    tmp_code_list = list(df.code.unique())
    tmp_code_list.sort()
    f_code, c_code, p_code = tmp_code_list
    code_list = [c_code, p_code, f_code]
    
    if strike%5!=0: strike += 0.5
    
    df.sort_values('time',inplace=True)
    df.index = range(len(df))
    df.time = pd.to_timedelta(df.time.str.replace('-','.'))
    df = df.replace(' ',0)
    df.code = df.code.astype('string')
    
    df.askc = df.askc.astype('float')
    df.askv = df.askv.astype('float')
    df.bidv = df.bidv.astype('float')
    df.bidc = df.bidc.astype('float')
    
    df['type'] = ''
    df.type = np.where( ( df.askv == 0 ) , 'ask side exec', df.type)
    df.type = np.where( ( df.bidv == 0 ) , 'bid side exec', df.type)
    df.type = np.where( (df.askc==0) & (df.bidc==0), 'price change', df.type )
    df.type = np.where( (df.type==''), 'volume change', df.type)
    
    df['exec_volume'] = np.nan
    df['exec_volume'] = np.where( df.type == 'ask side exec', df.bidc, df['exec_volume'] )
    df['exec_volume'] = np.where( df.type == 'bid side exec', df.askc, df['exec_volume'] )
    
    df['exec_price'] = np.nan
    df['exec_price'] = np.where(df.type == 'ask side exec', df.bidv, df['exec_price'])
    df['exec_price'] = np.where(df.type == 'bid side exec', df.askv, df['exec_price'])
    
    
    
    for name,code in zip(name_list,code_list):
        
        df[name+'askv'] = np.nan
        df[name+'askv'] = np.where((df.askv!=0) & (df.code == code) & 
                               (df.type != 'ask side exec') & (df.type != 'bid side exec') ,
                               df['askv'],df[name+'askv'])
        df[name+'askv'] = df[name+'askv'].ffill()
        
        df[name+'askp'] = np.nan
        df[name+'askp'] = np.where((df.askp!=0) & (df.code == code),df['askp'],df[name+'askp'])
        df[name+'askp'] = df[name+'askp'].ffill()
        
        df[name+'bidp'] = np.nan
        df[name+'bidp'] = np.where((df.bidp!=0) & (df.code == code),df['bidp'],df[name+'bidp'])
        df[name+'bidp'] = df[name+'bidp'].ffill()
        
        df[name+'bidv'] = np.nan
        df[name+'bidv'] = np.where((df.bidv!=0) & (df.code == code)  &
                               (df.type != 'ask side exec') & (df.type != 'bid side exec'),
                               df['bidv'],df[name+'bidv'])
        df[name+'bidv'] = df[name+'bidv'].ffill()
    
    df.exec_price.fillna(0,inplace = True)
    df.exec_volume.fillna(0, inplace = True)
    df.dropna(inplace = True)
    
    for name in name_list:
        
        df[name + 'askup'] = ''
        df[name + 'biddown'] = ''
        
        df[name + 'askup'] = np.where(df[name+'askp'] > df[name+'askp'].shift(1), 'O','')
        df[name + 'biddown'] = np.where(df[name + 'bidp'] < df[name+'bidp'].shift(1), 'O','')
    
    for name in name_list:
        #df[name+'hold'] = 0
        df[name+'buy_price'] = 0.0
        df[name+'sell_price'] = 0.0
    
    df['step'] = start
    df['end_of_loop'] = 0
    df['profit'] = 0.0
    
    strategy = strategy
    epoch = 1000000
    trade_size = 1
    profit_offset = 0.05
    
    df = df[(df.type == 'bid side exec') | (df.type == 'ask side exec')]
    df = df[(df.code == c_code )|(df.code == p_code)|(df.code == f_code)]
    df.index = range(len(df))
    
    for idx, row in df.iterrows():
        if idx == df.index.min() : continue
        #if idx % 200 == 0: print(idx)
        if idx>epoch: break
    
        for name in name_list:
            #df.loc[idx, name+'hold'] = df.loc[idx-1,name+'hold']
            df.loc[idx, name+'buy_price'] = df.loc[idx-1,name+'buy_price']
            df.loc[idx, name+'sell_price'] = df.loc[idx-1,name+'sell_price']
            
        df.loc[idx, 'step'] = df.loc[idx-1, 'step']
        
        if strategy == 0:
            # based on futures
            condition_zero_to_one = ( (df.loc[idx, c+'askp'] - df.loc[idx,p + 'bidp'] \
                -df.loc[idx,'exec_price']) <-margin1 -strike) and df.loc[idx,'type'] == 'ask side exec' \
                and df.loc[idx,'code'] == f_code and df.loc[idx-1, 'step'] == 0 and \
                    df.loc[idx,'exec_volume'] >= trade_size and df.loc[idx,c+'askv'] >= trade_size \
                        and df.loc[idx,p+'bidv'] >= trade_size
            
            condition_one_to_zero = ( (df.loc[idx, c+'bidp'] - df.loc[idx,p + 'askp'] \
                -df.loc[idx,'exec_price'] ) > margin2 -strike) and df.loc[idx,'type'] == 'bid side exec' \
                and df.loc[idx,'code'] == f_code and df.loc[idx-1, 'step'] == 2 and \
                    df.loc[idx,'exec_volume'] >= trade_size and df.loc[idx,c+'bidv'] >= trade_size \
                        and df.loc[idx,p+'askv'] >= trade_size
            
            if condition_zero_to_one:
                df.loc[idx,'step'] = 2
                df.loc[idx,c+'buy_price'] = df.loc[idx,c+'askp']
                df.loc[idx,p+'sell_price'] = df.loc[idx,p+'bidp']
                df.loc[idx,f+'sell_price'] = df.loc[idx,'exec_price']
                
                if start ==2:
                    for name in name_list:
                        df.loc[idx,'profit'] += multiplier * trade_size * (df.loc[idx,name+'sell_price'] \
                                                                   - df.loc[idx,name+'buy_price'])
                    
                    # 수수료 등 고려해서 profit 더 빼줌
                    df.loc[idx,'profit'] -= multiplier * trade_size * profit_offset
                    
                    for name in name_list:
                        df.loc[idx,name+'buy_price'] = 0
                        df.loc[idx,name+'sell_price'] = 0
                    
                    df.loc[idx,'end_of_loop'] = 1
                
            
            if condition_one_to_zero:
                df.loc[idx,'step'] = 0
                df.loc[idx,c+'sell_price'] = df.loc[idx,c+'bidp']
                df.loc[idx,p+'buy_price'] = df.loc[idx,p+'askp']
                df.loc[idx,f+'buy_price'] = df.loc[idx,'exec_price']
                
                if start ==0:
                    for name in name_list:
                        df.loc[idx,'profit'] += multiplier * trade_size * (df.loc[idx,name+'sell_price'] \
                                                                   - df.loc[idx,name+'buy_price'])
                    
                    # 수수료 등 고려해서 profit 더 빼줌
                    df.loc[idx,'profit'] -= multiplier * trade_size * profit_offset
                    
                    for name in name_list:
                        df.loc[idx,name+'buy_price'] = 0
                        df.loc[idx,name+'sell_price'] = 0
                    
                    df.loc[idx,'end_of_loop'] = 1
                    
                    
        
        elif strategy == 1:
            # based on call
            condition_zero_to_one = ( (df.loc[idx, 'exec_price'] - df.loc[idx,p + 'bidp'] \
                -df.loc[idx,f+'bidp'] ) <-margin1 -strike) and df.loc[idx,'type'] == 'bid side exec' \
                and df.loc[idx,'code'] == c_code and df.loc[idx-1, 'step'] == 0 and \
                    df.loc[idx,'exec_volume'] >= trade_size and df.loc[idx,p+'bidv'] >= trade_size \
                        and df.loc[idx,f+'bidv'] >= trade_size
            
            condition_one_to_zero = ( (df.loc[idx, 'exec_price'] - df.loc[idx,p + 'askp'] \
                -df.loc[idx,f+'askp'] ) > margin2 -strike) and df.loc[idx,'type'] == 'ask side exec' \
                and df.loc[idx,'code'] == c_code and df.loc[idx-1, 'step'] == 2 and \
                    df.loc[idx,'exec_volume'] >= trade_size and df.loc[idx,p+'askv'] >= trade_size \
                        and df.loc[idx,f+'askv'] >= trade_size
            
            if condition_zero_to_one:
                df.loc[idx,'step'] = 2
                df.loc[idx,c+'buy_price'] = df.loc[idx,'exec_price']
                df.loc[idx,p+'sell_price'] = df.loc[idx,p+'bidp']
                df.loc[idx,f+'sell_price'] = df.loc[idx,f+'bidp']
                
                if start ==2:
                    for name in name_list:
                        df.loc[idx,'profit'] += multiplier * trade_size * (df.loc[idx,name+'sell_price'] \
                                                                   - df.loc[idx,name+'buy_price'])
                    
                    # 수수료 등 고려해서 profit 더 빼줌
                    df.loc[idx,'profit'] -= multiplier * trade_size * profit_offset
                    
                    for name in name_list:
                        df.loc[idx,name+'buy_price'] = 0
                        df.loc[idx,name+'sell_price'] = 0
                    
                    df.loc[idx,'end_of_loop'] = 1
                    
                    
                    
            
            if condition_one_to_zero:
                df.loc[idx,'step'] = 0
                df.loc[idx,c+'sell_price'] = df.loc[idx,'exec_price']
                df.loc[idx,p+'buy_price'] = df.loc[idx,p+'askp']
                df.loc[idx,f+'buy_price'] = df.loc[idx,f+'askp']
                
                if start ==0:
                    for name in name_list:
                        df.loc[idx,'profit'] += multiplier * trade_size * (df.loc[idx,name+'sell_price'] \
                                                                   - df.loc[idx,name+'buy_price'])
                    
                    # 수수료 등 고려해서 profit 더 빼줌
                    df.loc[idx,'profit'] -= multiplier * trade_size * profit_offset
                    
                    for name in name_list:
                        df.loc[idx,name+'buy_price'] = 0
                        df.loc[idx,name+'sell_price'] = 0
                    
                    df.loc[idx,'end_of_loop'] = 1
                
                
        elif strategy == 2:
            # based on put
            condition_zero_to_one = ( (df.loc[idx, c+'askp'] - df.loc[idx,'exec_price'] \
                -df.loc[idx,f+'bidp']) <-margin1 -strike) and df.loc[idx,'type'] == 'ask side exec' \
                and df.loc[idx,'code'] == p_code and df.loc[idx-1, 'step'] == 0 and \
                    df.loc[idx,'exec_volume'] >= trade_size and df.loc[idx,c+'askv'] >= trade_size \
                        and df.loc[idx,f+'bidv'] >= trade_size
            
            condition_one_to_zero = ( (df.loc[idx, c+'bidp'] - df.loc[idx,'exec_price'] \
                -df.loc[idx,f+'askp']) > margin2 -strike) and df.loc[idx,'type'] == 'bid side exec' \
                and df.loc[idx,'code'] == p_code and df.loc[idx-1, 'step'] == 2 and \
                    df.loc[idx,'exec_volume'] >= trade_size and df.loc[idx,c+'bidv'] >= trade_size \
                        and df.loc[idx,f+'askv'] >= trade_size
            
            if condition_zero_to_one:
                df.loc[idx,'step'] = 2
                df.loc[idx,c+'buy_price'] = df.loc[idx,c+'askp']
                df.loc[idx,p+'sell_price'] = df.loc[idx,'exec_price']
                df.loc[idx,f+'sell_price'] = df.loc[idx,f+'bidp']
                
                if start ==2:
                    for name in name_list:
                        df.loc[idx,'profit'] += multiplier * trade_size * (df.loc[idx,name+'sell_price'] \
                                                                   - df.loc[idx,name+'buy_price'])
                    
                    # 수수료 등 고려해서 profit 더 빼줌
                    df.loc[idx,'profit'] -= multiplier * trade_size * profit_offset
                    
                    for name in name_list:
                        df.loc[idx,name+'buy_price'] = 0
                        df.loc[idx,name+'sell_price'] = 0
                    
                    df.loc[idx,'end_of_loop'] = 1
                
                
            
            if condition_one_to_zero:
                df.loc[idx,'step'] = 0
                df.loc[idx,c+'sell_price'] = df.loc[idx,c+'bidp']
                df.loc[idx,p+'buy_price'] = df.loc[idx,'exec_price']
                df.loc[idx,f+'buy_price'] = df.loc[idx,f+'askp']
                
                if start ==0:
                    for name in name_list:
                        df.loc[idx,'profit'] += multiplier * trade_size * (df.loc[idx,name+'sell_price'] \
                                                                   - df.loc[idx,name+'buy_price'])
                    
                    # 수수료 등 고려해서 profit 더 빼줌
                    df.loc[idx,'profit'] -= multiplier * trade_size * profit_offset
                    
                    for name in name_list:
                        df.loc[idx,name+'buy_price'] = 0
                        df.loc[idx,name+'sell_price'] = 0
                    
                    df.loc[idx,'end_of_loop'] = 1

    
    print(round((strike - 350)/2.5), ' date: ',the_date, 'start: ', start, 'strike: ',strike, 'strategy:', strategy, 'profit: ', int(sum(df.profit)), 'loop: ', sum(df.end_of_loop))
    return [df, strike, the_date, strategy, start , int(sum(df.profit)), sum(df.end_of_loop)]






#######################################################################################################################################################

the_date_list = ['20220223']
strike_list = [350,352,355,357,360,362,365,367,370,372,375]
strategy_list = [0,1,2]
start_list = [0,2]


df1 = []
df2 = []
df3 = []
df4 = []
df5 = []

# no limit order
for the_dates in the_date_list:
    for strikes in strike_list:
        df1.append(back_test1(strikes,the_dates,0.07,0))


# futures limit order
for the_dates in the_date_list:
    for strikes in strike_list:
        df2.append(back_test2(strikes,the_dates,0.07,0))


# call limit order
for the_dates in the_date_list:
    for strikes in strike_list:
        df3.append(back_test3(strikes,the_dates,0.07,0))


# put limit order
for the_dates in the_date_list:
    for strikes in strike_list:
        df4.append(back_test4(strikes,the_dates,0.07,0))


# loop based turnover rate
for strategies in strategy_list:
    for starts in start_list:
        for the_dates in the_date_list:
            for strikes in strike_list:
                df5.append(back_test5(strikes, the_dates, strategies , starts , 0.07, 0))
